In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


бостон - градиент уходит в 0  
ирис - спустя пару итераций в 0

In [3]:
# dataset = load_breast_cancer()
# dataset = load_boston()
dataset = load_wine()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
df = (df - df.mean())/(df.max() - df.min())
df0 = df.copy()
print(df.shape)
print(target)
df.head()

(178, 13)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.323522,-0.123784,0.033948,-0.200770,0.296287,0.174099,0.217454,-0.154441,0.220537,0.049651,0.067114,0.479236,0.226895
1,0.052469,-0.109950,-0.121132,-0.427574,0.002809,0.122375,0.154163,-0.192177,-0.098075,-0.057857,0.075244,0.288760,0.216196
2,0.041943,0.004674,0.162290,-0.046131,0.013679,0.174099,0.255428,-0.116706,0.384574,0.053064,0.058984,0.204511,0.312487
3,0.360364,-0.076353,0.071381,-0.138915,0.144113,0.536168,0.308171,-0.229913,0.185836,0.233951,-0.079227,0.307075,0.522901
4,0.062995,0.050129,0.269242,0.077580,0.198461,0.174099,0.139395,0.053106,0.072272,-0.062977,0.067114,0.116599,-0.008483


In [4]:
num_nan_cols = 2
nan_fraction = 0.3
nan_cols = np.random.random_integers(0, df.shape[1] - 1, num_nan_cols)
# print(df.isnull().mean())
for col in set(nan_cols):
    df.loc[df.sample(int(nan_fraction * len(df))).index, col] = np.nan
# print(df.isnull().mean())
# print(df.isnull().mean())

/Users/kek/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 12 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
nan_coords = np.array(np.where(df.isnull().values)).T
print('Num nan places: {}'.format(nan_coords.shape[0]))

Num nan places: 106


Считаем по каждой паре i_1 i_2 из coords градиент

In [6]:
df1 = df.loc[:, df.isnull().sum() == 0]
df2 = df.fillna(df.mean())
print(df1.shape, df2.shape)
arr0 = df0.values
arr1 = df1.values
arr2 = df2.values
print(arr0.shape, arr1.shape, arr2.shape)
print(np.isnan(arr0).sum(), np.isnan(arr1).sum(), np.isnan(arr2).sum())

(178, 11) (178, 13)
(178, 13) (178, 11) (178, 13)
0 0 0


In [8]:
arr2.shape

(178, 13)

# Пилим необходимые функции

df - исходная с прочерками    
df0 - копия исходной таблицы без прочерков  
df1 - сокращенная до ненулевых признаков  
df2 - исходная с восстановленными  
arr0,arr1,arr2 аналогично

In [8]:
# Алгоритм
# 1. Считаем эпсилоны (список)
# 2. Считаем оценки за маленькие классы
# 3. Выбираем интервал и дробим

In [7]:
n = len(df0)
def Cnk(n, k):
    a = b = c = tmp = 1
    for i in range(1, n+1):
        tmp *= i
        if i == n-k:
            a = tmp
        if i == k:
            b = tmp
        if i == n:
            c = tmp
    return c / (a*b)

In [24]:
def get_epsilons(data):
    def get_e(a):
        return np.abs(a - a[:, None]).mean()
    return np.array([get_e(feat) for feat in data.T])

In [47]:
def get_gamma(data, x, k, epsilons=None):
    '''
    необходимо вернуть кол-во пар values которые a<x<b
    a1 ... a_i < x < a_i+1 ... an
    '''
    if epsilons is not None:
        Gamma = 0
        for x_i in data:
            d = (np.abs(x_i - x) < epsilons).sum()
            Gamma += Cnk(d, k)
        return Gamma
    # расстояния между парами объектов   
    n = data.shape[0]
    n_feat = data.shape[1]
    maps = np.zeros(data.T.shape)
    for i in range(n_feat):
        maps[i] = data.T[i] <= x[i] 
    maps = maps.T
    Gamma = 0
    for i in tqdm(range(n)):
        for j in range(i, n):
            d = (maps[i] + maps[j] == 1).sum()
            Gamma += Cnk(d, k)
    return Gamma*2/(n*(n-1))

In [58]:
def predict(data, y, x, k, epsilons=None, scores=False):
    n_class = max(y) + 1
    g_classes = np.zeros(n_class)
    for i in range(n_class):
        c_data = data[y == i]
        g_classes[i] = get_gamma(c_data, x, k, epsilons)
    if scores:
        return g_classes
    return np.argmax(g_classes)

def predict_vect(X_train, y_train, X_test, k, epsilons=None, scores=False):
    return np.array([predict(X_train, y_train, x, k, epsilons, scores) for x in X_test])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(arr0, target, test_size=0.1, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
preds = rfc.predict(X_test)
preds

array([0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 1, 1])

In [13]:
eps = get_epsilons(X_train)

In [14]:
preds = np.array([predict(X_train, y_train, x, 3, eps) for x in X_test])

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

0.94444444444444442

# Восстановление прочерков с АВО

In [121]:
# итеративный процесс
def solve_avo(df, x, y):
    x_mask = df.iloc[:, y].isnull()
    y_mask = df.isnull().sum() > 0
    y_mask = y_mask[~y_mask].index

    X_train = df.iloc[x_mask[~x_mask].index, y_mask]
    X_test = df.iloc[x, y_mask].values.reshape(1, -1)

    y_train_raw = df.iloc[x_mask[~x_mask].index, y]
    y_train = np.argsort(y_train_raw)
    
    eps = get_epsilons(X_train.values)
    gammas_ki = predict(X_train.values, y_train, X_test, 3, eps, True)
    sizes_ki = y_train.value_counts().sort_index().values
    sort_index = solve_avo_gap(gammas_ki, sizes_ki, full_range)
    return y_train_raw.sort_values().iloc[sort_index]
    
def solve_avo_gap(gammas_ki, sizes_ki, full_range):
#     print(full_range)
    if len(full_range) == 1:
        return full_range[0]
    left_range = full_range[:full_range.size//2]
    right_range = full_range[full_range.size//2:]
    left_score = get_gamma_range(gammas_ki, sizes_ki, left_range)
    right_score = get_gamma_range(gammas_ki, sizes_ki, right_range)
    if left_score < right_score:
        return solve_avo_gap(gammas_ki, sizes_ki, right_range)
    else:
        return solve_avo_gap(gammas_ki, sizes_ki, left_range)
        
def get_gamma_range(gammas_ki, sizes_ki, y_range):
    sum_gammas = gammas_ki[y_range].sum()
    sum_sizes = sizes_ki[y_range].sum()
    if sum_sizes == 0:
        return 0
    return sum_gammas/sum_sizes

In [60]:
x_mask = df.iloc[:, y].isnull()
# x_mask = x_mask[~x_mask].index

y_mask = df.isnull().sum() > 0
y_mask = y_mask[~y_mask].index

X_train = df.iloc[x_mask[~x_mask].index, y_mask]
# X_test = df.iloc[x_mask[x_mask].index, y_mask]
X_test = df.iloc[x, y_mask].values.reshape(1, -1)

y_train_raw = df.iloc[x_mask[~x_mask].index, y]
y_train = np.argsort(y_train_raw)

# preds = predict_vect(X_train, y_train, X_test, 3)

In [51]:
# итерация распознавания
y_iter = (y_train > y_train.mean()).astype(int)
X_iter = X_train.values

In [54]:
eps = get_epsilons(X_train.values)
aa = predict_vect(X_iter, y_iter.values, X_test, 3, eps)[0]
# aa = predict_vect(X_iter, y_iter.values, X_test, 3, eps)[0]

In [ ]:
# выбрать класс для каждого уникального значения
# посчитать близость к каждому значению Гi
# далее делать бисекцию, в каждой операции сравнивать соотв взвешенные значения
def solve_partial_task(X_train, y_train_raw, X_test):
    y_iter = (y_train > y_train.mean()).astype(int)
    X_iter = X_train.values
    y_pred = predict_vect(X_iter, y_iter.values, X_test, 3)[0]
    y_train_raw = y_train_raw[y_iter == y_pred]
    X_train = X_train[y_iter == y_pred]
    
    

# Iterative process

In [ ]:
c = KL()
c

In [ ]:
mae = functions.get_mae(arr0, arr2, nan_coords)
mae

In [ ]:
cs = []
maes = []
msqes = []
accs = []

mae = functions.get_mae(arr0, arr2, nan_coords)
msqe = functions.get_msqe(arr0, arr2, nan_coords)
acc = functions.get_acc(arr2, target)

cs.append(c)
maes.append(mae)
msqes.append(msqe)
accs.append(acc)

In [ ]:
alpha = 0.1
max_iter = 30
print('\tIteration {} //  c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(0, c, mae, msqe, acc))
for i in range(max_iter):
    grad = get_full_grad(nan_coords)
    for j, (x,y) in enumerate(nan_coords):
        arr2[x, y] = arr2[x, y] - alpha*grad[j]
    P2_table = get_p2_table()
    c = KL()
    mae = functions.get_mae(arr0, arr2, nan_coords)
    msqe = functions.get_msqe(arr0, arr2, nan_coords)
    acc = functions.get_acc(arr2, target)
    
    accs.append(acc)
    cs.append(c)
    maes.append(mae)
    msqes.append(msqe)
    print('\tIteration {} // c {:.6} // mae {:.6} // rmse {:.6} // acc {}'.format(i+1, c,mae, 
                                                                                                   msqe, acc))